In [4]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 3. Encoder-Decoder Network for NMT

In [1]:
!pip install tensorflow-addons==0.9.1

In [3]:
vocab_size = 100
embed_size = 10

In [5]:
import tensorflow_addons as tfa

# inputs
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

# embeddings
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

# encoder
encoder = keras.layers.LSTM(512, return_state=True) # for carrying the last state to decoder
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

# decoder
sampler = tfa.seq2seq.sampler.TrainingSampler()
# sampling from the decoder output distribution 
# and producing the inputs for the next decoding step.
# at Training : previous target token
# at Inference : previous prediction token

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)#, training=None)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
# X = np.random.randint(100, size=10*1000).reshape(1000, 10)
# Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
# X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
# seq_lengths = np.full([1000], 15)

# history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

### 3.1 Bidirectional RNN
보통의 순환 층은 과거와 현재의 입력만 보고 출력을 생성한다. (단방향적인 causal model)  
이런 종류의 RNN은 시계열 예측시에는 적합하지만, nmt 등의 NLP TASK에는 맞지않음.  
단어는 해당 단어가 속한 문장 전체의 문맥에 영향을 받기 때문.
<br/><br/>
NLP에서 사용되는 RNN은 주로 Bidirectional하게 구현되는데,  
방향이 다른 unidirectional 층 2개를 사용하여 각 방향에 대해 학습시키고 concatenate 하여 사용한다.

In [7]:
# 구현은 단지 keras.layers.Bidirectional로 rnn layer를 감싸면 됨
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 10)          660       
_________________________________________________________________
bidirectional (Bidirectional (None, None, 20)          1320      
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


### 3.2 beam search
<br/>
decoder의 매 step 출력은 각 스텝에서 확률이 가장 높은 단어들을 출력하는 greedy한 방식이다.  
이 방식은 이후의 step까지 고려했을때 더 타당한 잠재적 정답들을 놓칠 수 있는 위험이 있다.
따라서 계산 리소스는 더 사용하나, 이런 위험을 피할 수 있는 search 알고리즘을 사용한다.  
<br/><br/>
모든 가능한 경우를 계산하는 exhaustive search는 안정적인 해답을 찾지만 계산량이 말도안된다.  
대신 실용적인 beam search가 널리 사용된다.
<img src = "https://d2l.ai/_images/beam-search.svg">

In [ ]:
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell = decoder_cell, beam_width = beam_width, output_layer = output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier = beam_width)
outputs, _, _ = decoder(
    embedding_decoder, start_tokens = start_tokens, end_token = end_token,
    initial_state = decoder_initial_state)